In [0]:
import pandas as pd
from google.colab import files 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
uploaded_training = files.upload()


Saving mnist_train.csv to mnist_train.csv


In [3]:
uploaded_testing = files.upload()


Saving mnist_test.csv to mnist_test.csv


In [4]:
for fn1 in uploaded_training.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn1, length=len(uploaded_training[fn1])))
for fn2 in uploaded_testing.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn2, length=len(uploaded_testing[fn2]))) 


User uploaded file "mnist_train.csv" with length 109991845 bytes
User uploaded file "mnist_test.csv" with length 18335285 bytes


In [0]:
import io 
train_df=pd.read_csv(io.StringIO(uploaded_training['mnist_train.csv'].decode('cp1252')),index_col=0)
test_df=pd.read_csv(io.StringIO(uploaded_testing['mnist_test.csv'].decode('cp1252')),index_col=0)

In [14]:
train_df.head(7)

,Category,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
Id,,,,,,,,,,,,,,,,,,,,,
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
test_df.head(2)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
Id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
train_df.shape

(60000, 785)

In [0]:
X_Train= (train_df.iloc[:,1:].values).astype('float32')
Y_Train=train_df['Category'].values
X_Test=test_df.values.astype('float32')

In [18]:
from keras.utils.np_utils import to_categorical 
Y_Train = to_categorical(Y_Train)
Y_Train.shape

(60000, 10)

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(X_Train, Y_Train, test_size = 0.10, random_state = 0)

In [20]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_validation = x_validation.reshape(-1, 28, 28, 1)
x_train.shape

(54000, 28, 28, 1)

In [21]:
x_train = x_train/255
x_validation = x_validation/255
x_validation.shape

(6000, 28, 28, 1)

In [22]:
print("x_train shape",x_train.shape)
print("x_validation shape",x_validation.shape)
print("y_train shape",y_train.shape)
print("y_validation shape",y_validation.shape)

x_train shape (54000, 28, 28, 1)
x_validation shape (6000, 28, 28, 1)
y_train shape (54000, 10)
y_validation shape (6000, 10)


In [0]:
from sklearn.metrics import confusion_matrix
import itertools
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization


In [0]:
#Better Performance
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal',input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.20))#0.20
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) #0.25
model.add(Conv2D(128, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(Dropout(0.25))#0.25
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))#0.25
model.add(Dense(10, activation='softmax'))

In [0]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)



In [0]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])



In [0]:
epochs = 50  # for better result increase the epochs epoch 30 and batch 64 for 99.02%
batch_size = 32

In [0]:
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False, 
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1, 
        horizontal_flip=False,  
        vertical_flip=False)  


In [0]:
datagen.fit(x_train)

In [30]:
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_validation,y_validation), steps_per_epoch=x_train.shape[0] // batch_size)

Epoch 1/50
1687/1687 [==============================] - 46s 27ms/step - loss: 0.2748 - acc: 0.9145 - val_loss: 0.0379 - val_acc: 0.9893
Epoch 2/50
1687/1687 [==============================] - 43s 25ms/step - loss: 0.0946 - acc: 0.9709 - val_loss: 0.0445 - val_acc: 0.9858
Epoch 3/50
1687/1687 [==============================] - 43s 26ms/step - loss: 0.0788 - acc: 0.9756 - val_loss: 0.0296 - val_acc: 0.9917
Epoch 4/50
1687/1687 [==============================] - 43s 25ms/step - loss: 0.0632 - acc: 0.9816 - val_loss: 0.0331 - val_acc: 0.9898
Epoch 5/50
1687/1687 [==============================] - 43s 25ms/step - loss: 0.0586 - acc: 0.9819 - val_loss: 0.0272 - val_acc: 0.9918
Epoch 6/50
1687/1687 [==============================] - 43s 25ms/step - loss: 0.0519 - acc: 0.9845 - val_loss: 0.0212 - val_acc: 0.9947
Epoch 7/50
1687/1687 [==============================] - 43s 25ms/step - loss: 0.0486 - acc: 0.9856 - val_loss: 0.0314 - val_acc: 0.9907
Epoch 8/50
1687/1687 [==========================

In [31]:
import statistics 
statistics.mean(history.history['val_acc']) #best submit 98.732

0.9945766666666667

In [32]:
x_test = X_Test.reshape(-1, 28, 28, 1)
x_test = x_test/255
print("x_test shape",x_test.shape)


x_test shape (10000, 28, 28, 1)


In [0]:
Y_Test= model.predict(x_test)

In [34]:
Y_Test.shape

(10000, 10)

In [0]:
results = np.argmax(Y_Test,axis = 1)
results = pd.Series(results,name="Label")

In [0]:
np.savetxt('testclass.csv', results, delimiter=',')

In [0]:
files.download('testclass.csv')